In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [5]:
import mynnlib
from mynnlib import *

dataset_dir = "insect-dataset"
models_dir = "models"

# Test

In [4]:
model_data = torch.load(f"{models_dir}/checkpoint.2025.02.16.butterfly.pth", weights_only=False)

In [6]:
test_top_k(model_data, f"{dataset_dir}/butterfly/random-test", 3, print_preds=False, print_top1_accuracy=True, print_no_match=False)

Top   1 accuracy: 113/153 -> 73.86%, genus matched: 139/153 -> 90.85%
Top   3 accuracy: 135/153 -> 88.24%, genus matched: 150/153 -> 98.04%


In [12]:
model = model_data['model']
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
scripted_model = torch.jit.script(model)

In [14]:
scripted_model.save(f"{models_dir}/m.checkpoint.2025.02.16.butterfly.pt")

# Convert all

In [19]:
import json

for model_name in ["2025.02.16.butterfly", "2025.02.16.moth", "2025.02.16.lepidoptera"]:
    model_data = torch.load(f"{models_dir}/checkpoint.{model_name}.pth", weights_only=False)
    model_data['model'].eval()
    scripted_model = torch.jit.script(model_data['model'])
    scripted_model.save(f"{models_dir}/mobile.checkpoint.{model_name}.pth")
    with open(f"{models_dir}/mobile.classes.{model_name}.json", "w", encoding="utf-8") as file:
        json.dump(model_data['class_names'], file, indent=4)